In [3]:
from IPython.display import YouTubeVideo

YouTubeVideo("RdhoN4VVqq8", width = 600, height=450)

## Generator by Example

In this example, we write some functions, `execute`, `worker` and `logger`, to demostrate the basic of generators.

- `execute`: a function which execute `worker` and do the logging by `logger`.
- `worker`: a function which do the job.
- `logger`: a function which will log out the current process.

In [1]:
# define the logger
def logger(prefix = "[Info]"):
    current_message = (yield) 
    while current_message is not None:
        print("{}: {}".format(prefix, current_message))
        current_message = (yield)
    
    print("process done")

Notice how we use the <code><font color='green'>yield</font></code> keyword.

The reason why we write our logger in such manner is that we will use the `send` method to pass message to our logger.

Let's see how it work

In [2]:
log = logger()
next(log) # start the logger (a generator)
log.send("hello")

[Info]: hello


The example above can be devided into following steps:

1. initiate the logger by invoke the logger generator
2. use next to start the logger and get it ready for `send`. At this time, the generator stop execution at `yield` keyword and wait for further information.
3. we send the message to the logger. At this time, the generator restart at where they stopped before and execute the code till the next `yield` or `return`.

In [3]:
next(log) # invoke it again

process done


StopIteration: 

Opps, a `StopIteration` error.

The reason for this is that there is no more item to generate. However, the final print will never be execute.

To fix this, we modify the `logger` generator a bit.

In [4]:
def logger2(prefix = "[Info]"):
    
    current_message = (yield) 
    while current_message is not None:
        print("{}: {}".format(prefix, current_message))
        current_message = (yield)
    print("process done")
    
    while True:
        yield # prevent StopIteration

In [5]:
log = logger2()
next(log)
log.send("Hello world")
next(log)

[Info]: Hello world
process done


In [6]:
# a simple worker
def worker(n):
    for i in range(n):
        yield i
    
    while True:
        yield

In [7]:
def execution(worker, logger):
    # start two generator
    next(logger)
    data = next(worker)
    while data is not None:
        logger.send("processing {}".format(data))
        data = next(worker)
    next(logger) # finish logging
    logger.close()

In [8]:
execution(worker(10), logger2())

[Info]: processing 0
[Info]: processing 1
[Info]: processing 2
[Info]: processing 3
[Info]: processing 4
[Info]: processing 5
[Info]: processing 6
[Info]: processing 7
[Info]: processing 8
[Info]: processing 9
process done


## Sending Multiple Values

In [43]:
def test():
    try:
        a, b = (yield)
        while a is not None or b is not None:
            print(a, b)
            try:
                a, b = (yield)
            except:
                pass
    except:
        pass
    
    while True:
        yield

In [44]:
t = test()

In [45]:
next(t)

In [46]:
t.send((1, 2))

1 2


In [47]:
t.send((3, 3))

3 3


In [48]:
t.send((None, None))

In [49]:
next(t)

In [50]:
t.close()

In [51]:
next(t)

StopIteration: 